# Scrap Data from KCDC

## Import

In [ ]:
import pandas as pd

import shutil
# importing os module
import os
# import datetime module
import datetime

import re

## Definitions

In [71]:
# your path folder to save results
PATH_FOLDER_SAVE = '../../data'

PATH_CSV_DATA = PATH_FOLDER_SAVE + '/data.csv'

PATH_PAGES_KCDC_UPDATES = os.getcwd() + '/pages_kcdc_updates.json'

PATH_TABLES_KCDC_UPDATES = os.getcwd() + '/tables_kcdc_updates.json'

## Helper functions

In [29]:
# save before scraping
def clean_file(path_file_name):
    '''
    Clean file already traited : rename file with date
    '''
    try:
        d = datetime.datetime.now()
        str_date = '_' + d.strftime("%Y%m%d_%H_%M_%S")
       
        res_re = re.search('\.\w+$', path_file_name)
        
        path_file_name_saved = \
            path_file_name[0:res_re.start()] + str_date + res_re.group(0)
         
        shutil.move(path_file_name, path_file_name_saved) 
        print('File {} moved!'.format(path_file_name_saved))
    except:
        print('File {} does not exist!'.format(path_file_name))


In [13]:
import scrapy
import scrapy.crawler as crawler
from multiprocessing import Process, Queue
from twisted.internet import reactor

# the wrapper to make it run more times
def run_spider(spider):
    '''
    DOESNT WORK WITH LAST VERSION OF SCRAPY OR TWISTED ??? 
    BUG
    function to run several times scraping process
    '''
    def f(q):
        try:
            runner = crawler.CrawlerRunner()
            deferred = runner.crawl(spider)
            deferred.addBoth(lambda _: reactor.stop())
            reactor.run()
            q.put(None)
        except Exception as e:
            q.put(e)

    q = Queue()
    p = Process(target=f, args=(q,))
    p.start()
    result = q.get()
    p.join()

    if result is not None:
        raise result


## Scrap pages links

In [50]:
%%writefile kcdcspider.py
# -*-coding:utf-8 -*

#
# Class to scrap KCDC Tables values for each town/area in South Korea
#

#import
import re
import scrapy

# definitions
URL_ROOT = 'https://www.cdc.go.kr'
NUM_MAX_PAGES = 4
PATH_OUTPUT = 'pages_kcdc_updates.json'
URL_FIRST_PAGE = URL_ROOT + '/board.es?mid=a30402000000&bid=0030&nPage=1'


# Spider Class
class KCDCPageSpider(scrapy.Spider):
    '''
    Spider to scrap all Le Gorafi pages from selected category
    Configure : 
    - KCDCPageSpider.custom_settings : save location 
    - num_max_pages : the number of next page to scrap
    - url_first_page : web page to start with
    '''
    name = "kcdc_updates_page"
    
    custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_OUTPUT
    }
    
    num_max_pages = NUM_MAX_PAGES
    
    url_first_page = URL_FIRST_PAGE
    
    def start_requests(self):
        urls = [
                self.url_first_page,
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)
    
    def parse(self, response):
        #//div[@class="dbody"]/ul/li/a[@title]/@href
        #//div[@class="dbody"]/ul/li/a[contains(@title,"updates")]/@href
        for post in response.xpath(
            '//div[@class="dbody"]/ul/li/a[contains(@title,"updates")]|' + \
            '//div[@class="dbody"]/ul/li/a[contains(@title,"Updates")]'):
            yield {
                'link': URL_ROOT + post.xpath('@href').extract_first()
            }
        # https://www.cdc.go.kr/board.es?mid=a30402000000&bid=0030&nPage=2
        #//a[@class="pageNext"]/@href
        next_page = response.xpath('//a[@class="pageNext"]/@href').get()
        
        if next_page is not None:
            try:
                num_next_page = int(re.search("(?<=\&nPage\=)\d+$", 
                         next_page).group(0))
                
                if (num_next_page < self.num_max_pages):
                    #next_page = response.urljoin(next_page)
                    next_page = URL_ROOT + next_page
                    yield scrapy.Request(next_page, callback=self.parse)
            except:
                next_page = None

Overwriting kcdcspider.py


In [51]:
%%writefile run_kcdc_pages_updates.py
# -*-coding:utf-8 -*

#
# Run scrapy for retrieve updates pages links from KCDC
#

# import
import scrapy
from scrapy.crawler import CrawlerProcess
from kcdcspider import KCDCPageSpider

process = CrawlerProcess()

process.crawl(KCDCPageSpider)
process.start() # the script will block here until the crawling is finished

Overwriting run_kcdc_pages_updates.py


In [53]:
%%time
try:
    # clean (move file if exist)
    clean_file(PATH_PAGES_KCDC_UPDATES)
    # execute external process (FALLBACK for scrapy problem)
    !python run_kcdc_pages_updates.py 
except:
    print("Error run_kcdc_pages_updates !")

2020-02-27 18:52:27 [scrapy.utils.log] INFO: Scrapy 1.8.0 started (bot: scrapybot)
2020-02-27 18:52:27 [scrapy.utils.log] INFO: Versions: lxml 4.5.0.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.10.0, Python 3.7.6 (default, Jan  8 2020, 13:42:34) - [Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.8, Platform Darwin-18.7.0-x86_64-i386-64bit
2020-02-27 18:52:27 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'json', 'FEED_URI': 'pages_kcdc_updates.json'}
2020-02-27 18:52:27 [scrapy.extensions.telnet] INFO: Telnet Password: e9523cd51050b96b
2020-02-27 18:52:27 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2020-02-27 18:52:27 [scrapy.middleware] INFO: Enabled downloader middlewares:
['sc

In [57]:
df_pages_kcdc_updates = pd.read_json(PATH_PAGES_KCDC_UPDATES)
df_pages_kcdc_updates

,link
0,https://www.cdc.go.kr/board/board.es?mid=a3040...
1,https://www.cdc.go.kr/board/board.es?mid=a3040...
2,https://www.cdc.go.kr/board/board.es?mid=a3040...
3,https://www.cdc.go.kr/board/board.es?mid=a3040...
4,https://www.cdc.go.kr/board/board.es?mid=a3040...
5,https://www.cdc.go.kr/board/board.es?mid=a3040...
6,https://www.cdc.go.kr/board/board.es?mid=a3040...
7,https://www.cdc.go.kr/board/board.es?mid=a3040...
8,https://www.cdc.go.kr/board/board.es?mid=a3040...
9,https://www.cdc.go.kr/board/board.es?mid=a3040...


In [49]:
# TODO ?
'''# configure scraping
KCDCPageSpider.url_first_page = URL_PAGES_KCDC_UPDATES
KCDCPageSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': URL_PAGES_KCDC_UPDATES
    }
KCDCPageSpider.num_max_pages = 1

# clean last output
clean_file(PATH_PAGES_KCDC_UPDATES)
# scraping page urls 
run_spider(KCDCPageSpider)'''
# DOES NOT WORK ANYMORE !


NameError: name 'URL_PAGES_KCDC_UPDATES' is not defined

## Scrap updates tables

In [127]:
%%writefile kcdctablespider.py
# -*-coding:utf-8 -*

#
# Module for scrapy over tables in KCDC
#

# import
import re
import scrapy
from scrapy.loader import ItemLoader
from scrapy.loader.processors import MapCompose, Join, TakeFirst
from w3lib.html import remove_tags


# definitions 
URL_ROOT = 'https://www.cdc.go.kr'
PATH_TABLES_OUTPUT = 'tables_kcdc_updates.json'
URL_FIRST_PAGE = URL_ROOT + '/board.es?mid=a30402000000&bid=0030&nPage=1'

LIST_AREA = ["Seoul",
"Busan",
"Daegu",
"Incheon",
"Gwangju",
"Daejeon",
"Ulsan",
"Sejong",
"Gyeonggi",
"Gangwon",
"Chungbuk",
"Chungnam",
"Cheonbuk",
"Cheonnam",
"Gyeongbuk",
"Gyeongnam",
"Jeju"]

def convert_int(str_in):
    # suppress ","
    str_in = re.sub(',', '', str_in)
    # convert in integer
    return int(str_in)

def find_in_table(response, n_row, n_col):
    search_str = '//tbody/tr/td/p/' + \
        'span[contains(., "Confirmed")]' + \
        '/../../../..//tr[{}]/td[{}]'.format(n_row, n_col) + \
        '/p/span[re:test(.,"^\d")]//text()'

    text_found = response.xpath(search_str).get()
    return text_found 




def find_int_in_table(response, n_row, n_col):
    text_found = find_in_table(response, n_row, n_col)
    if isinstance(text_found, str):
        return convert_int(text_found)
    else:
        return text_found


def find_int_area_in_table(response, area):

    search_str = '//tbody/tr/td/p/span[contains(., "{}")]'.format(area) + \
        '/../../../td[2]/p/span[re:test(.,"^\d")]//text()'
    text_found = response.xpath(search_str).get()
    
    if text_found is None:
        
        search_str = '//tbody/tr/td/p/span[contains(., "{}")]'.format(area) + \
        '/../../../../tr[3]/td[3]/p/span[re:test(.,"^\d")]//text()'
    
        text_found = response.xpath(search_str).get()
    
    if isinstance(text_found, str):
        return convert_int(text_found)
    else:
        return text_found
    

class KCDCiItem(scrapy.Item):
    '''
    Class item to declare different information to scrap
    and how to process (as input or output)
    '''
    # define the fields for your item here like:
    url = scrapy.Field(output_processor=TakeFirst())
    
    source = scrapy.Field(output_processor=TakeFirst())
    
    date_published = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=TakeFirst()
    )
    
    total = scrapy.Field(output_processor=TakeFirst())
    
    
    Seoul = scrapy.Field(output_processor=TakeFirst())   

    Busan = scrapy.Field(output_processor=TakeFirst())  
    Daegu = scrapy.Field(output_processor=TakeFirst())  
    Incheon = scrapy.Field(output_processor=TakeFirst())  
    Gwangju = scrapy.Field(output_processor=TakeFirst())  
    Daejeon = scrapy.Field(output_processor=TakeFirst())  
    Ulsan = scrapy.Field(output_processor=TakeFirst())  
    Sejong = scrapy.Field(output_processor=TakeFirst())  
    Gyeonggi = scrapy.Field(output_processor=TakeFirst())  
    Gangwon = scrapy.Field(output_processor=TakeFirst())  
    Chungbuk = scrapy.Field(output_processor=TakeFirst())  
    Chungnam = scrapy.Field(output_processor=TakeFirst())  
    Cheonbuk = scrapy.Field(output_processor=TakeFirst())  
    Cheonnam = scrapy.Field(output_processor=TakeFirst())  
    Gyeongbuk = scrapy.Field(output_processor=TakeFirst())  
    Gyeongnam = scrapy.Field(output_processor=TakeFirst())  
    Jeju = scrapy.Field(output_processor=TakeFirst())  
    

class KCDCTablesSpider(scrapy.Spider):
    '''
    Spider to scrap tables webpages : 
    - how to find information for scraping
    - which field names to store
    '''
    # Your spider definition
    name = 'tables_kcdc_spider'
    # output definition :
    custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_TABLES_OUTPUT
  }
    # urls to scrap
    start_urls = [URL_FIRST_PAGE]
   
    def parse(self, response):
        '''
        Parse definition with xpath which define all patterns to use
        for retrieve information into HTML strings
        '''
        #url	source	author	title	theme	description	date_published	body   
        l = ItemLoader(item=KCDCiItem(), selector=response)
        
        l.add_value('url', response.url)
        
        l.add_value('source', "KCDC")
        
        #l.add_xpath('author', 
        #    "//section[@class='metas container']/span[@class='context']/a")
        
        #l.add_xpath("title", "//h1")
        
        #l.add_xpath("theme", "//a[@rel='category tag']")
        
        #l.add_xpath("description", ".//div[@class='intro']/p")
        
        #l.add_xpath("date_published", 
        #    "//section[@class='metas container']/span[@class='context']",
        #    re="[0-9]+/[0-9]+/[0-9]+")
        
        l.add_xpath("date_published", 
                    '//span[re:test(.,"^Update$")]/../b//text()', 
                    re="[0-9]+-[0-9]+-[0-9]+")
        
        #response.xpath('//tbody/tr/td/p/span[contains(., "Total")]/../../../td[2]/p/span[re:test(.,"^\d")]//text()').get() 
        #flag_total = response.xpath('//tbody/tr/td/p/' + \
        #    'span[contains(., "Confirmed")]' + \
        #    '/../../../..//tr[4]/td[3]/p/span[re:test(.,"^\d")]//text()').get()
        
        # check if more than 1 row of data
        flag_total = find_in_table(response, 4, 3)
        
        if flag_total is not None: # take 3rd row
            '''text_2 = response.xpath('//tbody/tr/td/p/' + \
                'span[contains(., "Confirmed")]' + \
                '/../../../..//tr[3]/td[3]' + \
                '/p/span[re:test(.,"^\d")]//text()').get()
            # convert in integer
            int_2 = convert_int(text_2)
            
            text_3 = response.xpath('//tbody/tr/td/p/' + \
                'span[contains(., "Confirmed")]' + \
                '/../../../..//tr[4]/td[3]' + \
                '/p/span[re:test(.,"^\d")]//text()').get()
            int_3 = convert_int(text_3)'''
            
            
            flag_sub = response.xpath('//tbody/tr/td/p/' + \
                'span[contains(., "Confirmed")]/../../../..//tr[2]/td[1]' + \
                '/p/span[re:test(.,"^Sub")]//text()').get()
            
            # if sub total exist (don't sum up)
            if flag_sub is not None:
                int_total_2 = find_int_in_table(response, 3, 3)
                int_total_3 = find_int_in_table(response, 4, 3)
            
            else:
                int_2_1 = find_int_in_table(response, 3, 3)
                int_2_2 = find_int_in_table(response, 3, 4)
                int_total_2 = int_2_1 + int_2_2
                int_3_1 = find_int_in_table(response, 4, 3)
                int_3_2 = find_int_in_table(response, 4, 4)
                int_total_3 = int_3_1 + int_3_2
            
            if int_total_3 > int_total_2:
                l.add_value("total", int_total_3)
            else:
                l.add_value("total", int_total_2)
            
            '''l.add_xpath("total", '//tbody/tr/td/p/' + \
                'span[contains(., "Confirmed")]' + \
                '/../../../..//tr[4]/td[3]/p/span[re:test(.,"^\d")]//text()')'''
            
        else: # take 2nd row
            int_total = find_int_in_table(response, 3, 3)
            l.add_value("total", int_total)
            
            '''l.add_xpath("total", '//tbody/tr/td/p/' + \
                'span[contains(., "Confirmed")]' + \
                '/../../../..//tr[3]/td[3]/p/span[re:test(.,"^\d")]//text()')'''
        
        for area in LIST_AREA:
            int_area = find_int_area_in_table(response, area)
            if int_area is not None:
                l.add_value(area, int_area)
        
        '''l.add_xpath("Seoul", 
            '//tbody/tr/td/p/span[contains(., "Seoul")]' + \
                    '/../../../td[2]/p/span[re:test(.,"^\d")]//text()')'''
        
        
        #l.add_xpath("body", "//div[@class='content']/p")
        #l.add_xpath("body", 
        #    '//div[@class="content"]/p[not(script)][not(img)][not(video)][not(strong/a)]//text()')
        
        yield l.load_item()


Overwriting kcdctablespider.py


In [113]:
isinstance("345", str)

True

In [114]:
'//tbody/tr/td/p/span[contains(., "{}")]' + \
        '/../../../td[2]/p/span[re:test(.,"^\d")]//text()'.format("Seoul")

'//tbody/tr/td/p/span[contains(., "{}")]/../../../td[2]/p/span[re:test(.,"^\\d")]//text()'

In [94]:
int(re.sub(',', '', '1,567'))

1567

In [84]:
# response.xpath('//tbody/tr/td/p/span[contains(., "Total")]/../../../td[2]/p/span[re:test(.,"^\d")]//text()').get()  


In [ ]:
'//tbody/tr/td/p/span[contains(., "Seoul"]/../../../td[2]/p/span[re:test(.,"^\d")]//text()'

In [ ]:
# //tbody/tr/td/p/span[contains(., "Seoul")]/../../../../tr[3]/td[3]/p/span

In [100]:
%%writefile run_kcdc_tables_updates.py
# -*-coding:utf-8 -*

#
# Run scrapy for retrieve updates tables from KCDC
#

# import
import scrapy
from scrapy.crawler import CrawlerProcess
from kcdctablespider import KCDCTablesSpider
import pandas as pd
import os

# definitions
PATH_PAGES_KCDC_UPDATES = os.getcwd() + '/pages_kcdc_updates.json'

# get urls pages links
df_pages_kcdc_updates = pd.read_json(PATH_PAGES_KCDC_UPDATES)
# define url for scrapy
KCDCTablesSpider.start_urls = df_pages_kcdc_updates["link"].tolist()

# start scraping process
process = CrawlerProcess()
process.crawl(KCDCTablesSpider)
process.start() # the script will block here until the crawling is finished

Overwriting run_kcdc_tables_updates.py


In [128]:
%%time
try:
    # clean (move file if exist)
    clean_file(PATH_TABLES_KCDC_UPDATES)
    # execute external process (FALLBACK for scrapy problem)
    !python run_kcdc_tables_updates.py 
except:
    print("Error run_kcdc_tables_updates !")

File /Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/tables_kcdc_updates_20200228_11_33_14.json moved!
2020-02-28 11:33:15 [scrapy.utils.log] INFO: Scrapy 1.8.0 started (bot: scrapybot)
2020-02-28 11:33:15 [scrapy.utils.log] INFO: Versions: lxml 4.5.0.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.10.0, Python 3.7.6 (default, Jan  8 2020, 13:42:34) - [Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 19.1.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.8, Platform Darwin-18.7.0-x86_64-i386-64bit
2020-02-28 11:33:15 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'json', 'FEED_URI': 'tables_kcdc_updates.json'}
2020-02-28 11:33:15 [scrapy.extensions.telnet] INFO: Telnet Password: ba4ea96cc1d5f02f
2020-02-28 11:33:15 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.exte

2020-02-28 11:33:21 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.cdc.go.kr/board/board.es?mid=a30402000000&bid=0030&act=view&list_no=366261&tag=&nPage=3>
{'date_published': '2020-02-20',
 'source': 'KCDC',
 'total': 82,
 'url': 'https://www.cdc.go.kr/board/board.es?mid=a30402000000&bid=0030&act=view&list_no=366261&tag=&nPage=3'}
2020-02-28 11:33:21 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.cdc.go.kr/board/board.es?mid=a30402000000&bid=0030&act=view&list_no=366299&tag=&nPage=2> (referer: None)
2020-02-28 11:33:21 [scrapy.core.scraper] DEBUG: Scraped from <200 https://www.cdc.go.kr/board/board.es?mid=a30402000000&bid=0030&act=view&list_no=366299&tag=&nPage=2>
{'date_published': '2020-02-22',
 'source': 'KCDC',
 'total': 346,
 'url': 'https://www.cdc.go.kr/board/board.es?mid=a30402000000&bid=0030&act=view&list_no=366299&tag=&nPage=2'}
2020-02-28 11:33:22 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.cdc.go.kr/board/board.es?mid=a30402000000&bid=

In [129]:
PATH_TABLES_KCDC_UPDATES

'/Users/gregory/Documents/CloudStationSinchon/Applications/python/CoronaVirus/code/coronavirusModel/tables_kcdc_updates.json'

In [131]:
df_tables_kcdc_updates = pd.read_json(PATH_TABLES_KCDC_UPDATES)
df_tables_kcdc_updates.T

,0,1,2,3,4,5,6,7,8,9,10,11,12
url,https://www.cdc.go.kr/board/board.es?mid=a3040...,https://www.cdc.go.kr/board/board.es?mid=a3040...,https://www.cdc.go.kr/board/board.es?mid=a3040...,https://www.cdc.go.kr/board/board.es?mid=a3040...,https://www.cdc.go.kr/board/board.es?mid=a3040...,https://www.cdc.go.kr/board/board.es?mid=a3040...,https://www.cdc.go.kr/board/board.es?mid=a3040...,https://www.cdc.go.kr/board/board.es?mid=a3040...,https://www.cdc.go.kr/board/board.es?mid=a3040...,https://www.cdc.go.kr/board/board.es?mid=a3040...,https://www.cdc.go.kr/board/board.es?mid=a3040...,https://www.cdc.go.kr/board/board.es?mid=a3040...,https://www.cdc.go.kr/board/board.es?mid=a3040...
source,KCDC,KCDC,KCDC,KCDC,KCDC,KCDC,KCDC,KCDC,KCDC,KCDC,KCDC,KCDC,KCDC
date_published,2020-02-20,2020-02-23,2020-02-24,2020-02-25,2020-02-26,2020-02-20,2020-02-18,2020-02-17,2020-02-19,2020-02-20,2020-02-22,2020-02-27,2020-02-21
total,104,556,763,893,1146,82,31,30,46,82,346,1595,156
Seoul,NaN,NaN,30,36,45,NaN,NaN,NaN,NaN,NaN,NaN,55,4
Busan,NaN,NaN,17,38,50,NaN,NaN,NaN,NaN,NaN,NaN,58,NaN
Daegu,NaN,NaN,442,499,677,NaN,NaN,NaN,1996,NaN,NaN,1017,4
Incheon,NaN,NaN,2,2,3,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN
Gwangju,NaN,NaN,9,9,9,NaN,NaN,NaN,NaN,NaN,NaN,9,4
Daejeon,NaN,NaN,3,3,3,NaN,NaN,NaN,NaN,NaN,NaN,8,NaN


In [90]:
df_tables_kcdc_updates["url"].tolist()

['https://www.cdc.go.kr/board/board.es?mid=a30402000000&bid=0030&act=view&list_no=366299&tag=&nPage=2',
 'https://www.cdc.go.kr/board/board.es?mid=a30402000000&bid=0030&act=view&list_no=366264&tag=&nPage=3',
 'https://www.cdc.go.kr/board/board.es?mid=a30402000000&bid=0030&act=view&list_no=366307&tag=&nPage=2',
 'https://www.cdc.go.kr/board/board.es?mid=a30402000000&bid=0030&act=view&list_no=366328&tag=&nPage=1',
 'https://www.cdc.go.kr/board/board.es?mid=a30402000000&bid=0030&act=view&list_no=366350&tag=&nPage=1',
 'https://www.cdc.go.kr/board/board.es?mid=a30402000000&bid=0030&act=view&list_no=366338&tag=&nPage=1',
 'https://www.cdc.go.kr/board/board.es?mid=a30402000000&bid=0030&act=view&list_no=366363&tag=&nPage=1',
 'https://www.cdc.go.kr/board/board.es?mid=a30402000000&bid=0030&act=view&list_no=366288&tag=&nPage=2',
 'https://www.cdc.go.kr/board/board.es?mid=a30402000000&bid=0030&act=view&list_no=366255&tag=&nPage=3',
 'https://www.cdc.go.kr/board/board.es?mid=a30402000000&bid=0030